Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [4]:
x = tf.Variable([[1., 1.], [2., 2.]])
y = tf.Variable([1., 1. , 5])
loss = tf.nn.l2_loss([x])
sess = tf.Session()
with sess.as_default():
    tf.global_variables_initializer().run()
    print(x.eval())   # equivalent to session.run(b)
    print(loss.eval())


[[1. 1.]
 [2. 2.]]
5.0


In [3]:
x

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32_ref>

First reload the data we generated in `1_notmnist.ipynb`.

In [5]:
data_root = '/home/roman/data/nomnist' # Change me to store data elsewhere
pickle_file = data_root + '/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.005
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  #tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  #tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  # Matrix W of size KxY, where K is number of inputs, Y is number of outputs 
  # (in 1-hot encoding for multinomial regression)
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))  # Intercept
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  if beta > 0:
    loss += beta * (tf.nn.l2_loss(weights)) #  + tf.nn.l2_loss(biases)
    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3700

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 33.523209
Training accuracy: 4.7%
Validation accuracy: 11.9%
Loss at step 500: 1.754587
Training accuracy: 84.4%
Validation accuracy: 79.7%
Loss at step 1000: 0.762139
Training accuracy: 81.2%
Validation accuracy: 81.2%
Loss at step 1500: 0.678282
Training accuracy: 81.2%
Validation accuracy: 81.2%
Loss at step 2000: 0.565775
Training accuracy: 85.9%
Validation accuracy: 80.7%
Loss at step 2500: 0.688435
Training accuracy: 82.0%
Validation accuracy: 81.5%
Loss at step 3000: 0.642090
Training accuracy: 83.6%
Validation accuracy: 81.2%
Loss at step 3500: 0.823383
Training accuracy: 78.9%
Validation accuracy: 80.9%
Test accuracy: 88.1%


In [8]:
# Simple regularization
batch_size = 256
hidden_size = 1024
input_size = image_size * image_size

def setup_lt(size_vec):
    assert(size_vec)
    res = []
    for index in range(len(size_vec) - 1):
        w = tf.Variable(tf.truncated_normal([size_vec[index], size_vec[index+1]]))
        b = tf.Variable(tf.zeros([size_vec[index+1]]))
        res.append((w, b))
    return res

def forward_prop(input, weights, apply_softmax = False):
    assert(weights)
    prev = input
    for index in range(len(weights) - 1):
      prev = tf.nn.relu(tf.matmul(prev, weights[index][0]) + weights[index][1])
    
    o = tf.matmul(prev, weights[-1][0]) + weights[-1][1]
    if apply_softmax:
        o = tf.nn.softmax(o)
    return o
    
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, input_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_beta = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = setup_lt((input_size, hidden_size, num_labels))

  # Training computation.
  logits = forward_prop(tf_train_dataset, weights)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  for w in weights:
    loss += tf_beta * (tf.nn.l2_loss(w[0]) + tf.nn.l2_loss(w[1]))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_prediction = forward_prop(tf_valid_dataset, weights, apply_softmax=True) 
  test_prediction = forward_prop(tf_test_dataset, weights, apply_softmax=True)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [9]:
num_steps = 4700
beta = 0.0005

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_beta : beta}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 473.281799
Training accuracy: 11.3%
Validation accuracy: 31.8%
Loss at step 500: 127.646797
Training accuracy: 84.8%
Validation accuracy: 81.4%
Loss at step 1000: 99.154327
Training accuracy: 82.4%
Validation accuracy: 81.8%
Loss at step 1500: 75.001366
Training accuracy: 85.2%
Validation accuracy: 82.7%
Loss at step 2000: 57.165184
Training accuracy: 84.0%
Validation accuracy: 84.5%
Loss at step 2500: 44.680801
Training accuracy: 83.6%
Validation accuracy: 84.7%
Loss at step 3000: 34.665627
Training accuracy: 85.5%
Validation accuracy: 85.7%
Loss at step 3500: 26.704750
Training accuracy: 89.5%
Validation accuracy: 85.7%
Loss at step 4000: 20.900681
Training accuracy: 90.6%
Validation accuracy: 86.2%
Loss at step 4500: 16.413078
Training accuracy: 89.5%
Validation accuracy: 87.1%
Test accuracy: 93.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [23]:
num_steps = 4000
beta = 0.0005

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step % 100) * batch_size 
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_beta : beta}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 582.985718
Training accuracy: 4.3%
Validation accuracy: 28.0%
Loss at step 500: 124.989838
Training accuracy: 88.3%
Validation accuracy: 80.8%
Loss at step 1000: 94.137672
Training accuracy: 97.3%
Validation accuracy: 82.2%
Loss at step 1500: 73.200073
Training accuracy: 99.6%
Validation accuracy: 83.0%
Loss at step 2000: 57.102257
Training accuracy: 98.4%
Validation accuracy: 82.8%
Loss at step 2500: 44.418251
Training accuracy: 99.2%
Validation accuracy: 83.5%
Loss at step 3000: 34.598328
Training accuracy: 99.6%
Validation accuracy: 83.5%
Loss at step 3500: 26.944874
Training accuracy: 100.0%
Validation accuracy: 83.8%
Test accuracy: 90.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [40]:
#Regularization + single dropout before output layer + Xavier weight initialization
batch_size = 256
hidden_size = 1024
input_size = image_size * image_size

def setup_lt(size_vec):
    assert(size_vec)
    res = []
    for index in range(len(size_vec) - 1):
        # Use Xavier initialiation for weigths
        input_sz = size_vec[index]
        output_sz = size_vec[index+1]
        w = tf.Variable(tf.truncated_normal([input_sz, output_sz], stddev = np.sqrt(2.0 /float(input_sz))))
        b = tf.Variable(tf.zeros([output_sz]))
        res.append((w, b))
    return res


def forward_prop(input, weights, apply_softmax = False, dropout=None):
    assert(weights)
    prev = input
    for index in range(len(weights) - 1):
      prev = tf.nn.relu(tf.matmul(prev, weights[index][0]) + weights[index][1])
    if dropout:
        prev = tf.nn.dropout(prev, keep_prob = (1 - dropout))
        
    o = tf.matmul(prev, weights[-1][0]) + weights[-1][1]
    if apply_softmax:
        o = tf.nn.softmax(o)
    return o


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, input_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_beta = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = setup_lt((input_size, 1024, 300, 75, num_labels))

  # Training computation.
  logits = forward_prop(tf_train_dataset, weights, dropout=None)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  for w in weights:
    loss += tf_beta * (tf.nn.l2_loss(w[0]) + tf.nn.l2_loss(w[1]))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  learning_rate = tf.train.exponential_decay(0.1, global_step,
                                             500, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_prediction = forward_prop(tf_valid_dataset, weights, apply_softmax=True) 
  test_prediction = forward_prop(tf_test_dataset, weights, apply_softmax=True)

In [41]:
num_steps = 9000
beta = 0.0001

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_beta : beta}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Loss at step %d: %f, learning rate: %f' % (step, l, learning_rate.eval()))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, batch_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 2.593709, learning rate: 0.100000
Training accuracy: 9.4%
Validation accuracy: 19.0%
Loss at step 500: 0.582199, learning rate: 0.096000
Training accuracy: 86.7%
Validation accuracy: 85.7%
Loss at step 1000: 0.495599, learning rate: 0.092160
Training accuracy: 88.7%
Validation accuracy: 87.5%
Loss at step 1500: 0.507626, learning rate: 0.088474
Training accuracy: 87.9%
Validation accuracy: 88.3%
Loss at step 2000: 0.435574, learning rate: 0.084935
Training accuracy: 91.4%
Validation accuracy: 88.6%
Loss at step 2500: 0.457279, learning rate: 0.081537
Training accuracy: 87.9%
Validation accuracy: 88.7%
Loss at step 3000: 0.472461, learning rate: 0.078276
Training accuracy: 89.5%
Validation accuracy: 89.3%
Loss at step 3500: 0.368832, learning rate: 0.075145
Training accuracy: 92.2%
Validation accuracy: 89.6%
Loss at step 4000: 0.374699, learning rate: 0.072139
Training accuracy: 92.6%
Validation accuracy: 89.3%
Loss at step 4500: 0.427227, learning rate: 0.06

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
